<a href="https://colab.research.google.com/github/akshala/POS-tagging-using-HMM-and-viterbi-algorithm/blob/main/hmm_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Brown_train.txt to Brown_train (2).txt


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from statistics import mean
from sklearn.model_selection import KFold
import numpy as np

In [ ]:
with open('Brown_train.txt') as f:
  text = f.read()
# text

In [ ]:
sentences = text.split('\n')
sentences = ['<s>_<s> <s>_<s> ' + elt for elt in sentences if len(elt) > 0]
# sentences

In [ ]:
final = []
all_words = []
for sentence in sentences:
  if len(sentence) == 0:
    continue
  words = sentence.split()
  all_words.extend(words)
  word_tag_pair = []
  for word in words:
    word = word.split('_')
    try:
      word_tag_pair.append((word[0], word[1]))
    except IndexError:
      pass
  final.append(word_tag_pair)
# final
# len(all_words)

In [ ]:
sentence_lengths = []
for sentence in sentences:
  sentence_lengths.append(len(sentence))
# print('max sentence length', max(sentence_lengths))
# print('min sentence length', min(sentence_lengths))
# print('avg sentence length', mean(sentence_lengths))
# print('total number of sentences', len(sentence_lengths))

In [ ]:
vocab_word_tag = {}
tag_word_dict = {}
for sentence in final:
  for pair in sentence:
    word = pair[0].lower()
    tag = pair[1]
    if word in vocab_word_tag:
      vocab_word_tag[word].add(tag)
    else:
      vocab_word_tag[word] = set()
      vocab_word_tag[word].add(tag)
    if tag in tag_word_dict:
      tag_word_dict[tag].add(word)
    else:
      tag_word_dict[tag] = set()
      tag_word_dict[tag].add(word)
# print('word key tag value', len(vocab_word_tag))
# print('tag key word value', len(tag_word_dict))

In [ ]:
vocab = list(vocab_word_tag.keys())
vocab.sort()
# len(vocab)
# vocab

In [ ]:
tags = list(tag_word_dict.keys())
tags.sort()
# len(tags)
# tags

In [ ]:
def get_tag_vocab(input_array):
  vocab_word_tag = {}
  tag_word_dict = {}
  for sentence in input_array:
    for pair in sentence:
      word = pair[0].lower()
      tag = pair[1]
      if word in vocab_word_tag:
        vocab_word_tag[word].add(tag)
      else:
        vocab_word_tag[word] = set()
        vocab_word_tag[word].add(tag)
      if tag in tag_word_dict:
        tag_word_dict[tag].add(word)
      else:
        tag_word_dict[tag] = set()
        tag_word_dict[tag].add(word)
  vocab = list(vocab_word_tag.keys())
  vocab.append('<unknown>')
  vocab.sort()
  tags = list(tag_word_dict.keys())
  tags.sort()
  vocab_word_tag['<unknown>'] = set()
  for elt in tags:
    vocab_word_tag['<unknown>'].add(elt)
  return vocab, tags, vocab_word_tag

In [ ]:
num_words_for_tag = {}
for key, val in tag_word_dict.items():
  num_words_for_tag[key] = len(val)
# num_words_for_tag

In [ ]:
total_words_vocab = vocab_word_tag.keys()
# len(total_words_vocab)

In [ ]:
def get_counts(input_array):
  transition_counts = {}
  emission_counts = {}
  tag_counts = {}
  for sentence in input_array:
    prev1_tag = '<s>'
    prev2_tag = '<s>'
    for pair in sentence:
      word = pair[0].lower()
      tag = pair[1]
      if (tag, prev1_tag, prev2_tag) not in transition_counts:
        transition_counts[(tag, prev1_tag, prev2_tag)] = 1
      else:
        transition_counts[(tag, prev1_tag, prev2_tag)] += 1
      if (word, tag) not in emission_counts:
        emission_counts[(word, tag)] = 1
      else:
        emission_counts[(word, tag)] += 1
      if tag not in tag_counts:
        tag_counts[tag] = 1
      else:
        tag_counts[tag] += 1
      prev2_tag = prev1_tag
      prev1_tag = tag
  tag_counts['<unknown>'] = 0
  return transition_counts, emission_counts, tag_counts

In [ ]:
def get_2_tag_counts(input_array):
  two_tag_counts = {}
  for sentence in input_array:
    prev = '<s>'
    i = 0
    for pair in sentence:
      tag = pair[1]
      if i != 0:
        if (tag, prev) not in two_tag_counts:
          two_tag_counts[(tag, prev)] = 1
        else:
          two_tag_counts[(tag, prev)] += 1
      i += 1
      prev = tag
  return two_tag_counts

In [ ]:
def get_transition_matrix(transition_counts, two_tag_counts, tags):
  transition_prob = {}
  for key, val in transition_counts.items():
    prev1_tag = key[1]
    prev2_tag = key[2]
    transition_prob[key] = val/two_tag_counts[(prev1_tag, prev2_tag)]
  return transition_prob

In [ ]:
def get_emission_matrix(emission_counts, tag_counts, tags, vocab):
  emission_prob = {}
  total_tag_count = sum(list(tag_counts.values()))
  for key, val in emission_counts.items():
    tag = key[1]
    emission_prob[key] = val/tag_counts[tag]
  for key, val in tag_counts.items():
    word = '<unknown>'
    emission_prob[(word, key)] = val/total_tag_count
  return emission_prob

In [ ]:
def viterbi(transition_probs, emission_probs, test, vocab_word_tag):
  prev1_word = '<s>'
  prev2_word = '<s>'
  predicted = []
  count = 1
  for word in test:
    prob = []
    prob_tag = []
    for prev1_tag in vocab_word_tag[prev1_word]:
      for prev2_tag in vocab_word_tag[prev2_word]:
        try:
          tag_list = vocab_word_tag[word]
        except KeyError:
          tag_list = vocab_word_tag['<unknown>']
          word = '<unknown>'
        for cur_tag in tag_list:
          try:
            if word == '<unknown>':
              prob.append(np.log(transition_probs[(cur_tag, prev1_tag, prev2_tag)]))
            else:
              prob.append(np.log(transition_probs[(cur_tag, prev1_tag, prev2_tag)]) + np.log(emission_probs[(word, cur_tag)]))
          except KeyError:
            prob.append(np.log(emission_probs[(word, cur_tag)]))
          prob_tag.append(cur_tag)
      prev2_word = prev1_word
      prev1_word = word
    n = len(prob)
    max_prob = float('-inf')
    max_prob_tag = ''
    for i in range(n):
      if prob[i] > max_prob:
        max_prob = prob[i]
        max_prob_tag = prob_tag[i]
      splitted = max_prob_tag.split('-')
      if splitted[0] == 'FW':
        predicted.append(splitted[1]) 
      else: 
        predicted.append(splitted[0])
    # print(count, max_prob_tag)
    count += 1
  return predicted

In [ ]:
def word_tag_seperate(word_tag_pair_sentences):
  words = []
  tags = []
  for sentence in word_tag_pair_sentences:
    sentence_words = []
    sentences_tags = []
    for pair in sentence:
      words.append(pair[0].lower())
      tags.append(pair[1])
  return words, tags

In [4]:
kFold = KFold(n_splits=3)
predicted_tags = []
all_test_tags = []
for train, test in kFold.split(final):
  training_set = []
  testing_set = []
  for elt in train:
    training_set.append(final[elt])
  for elt in test:
    testing_set.append(final[elt])
  test_words, test_tags = word_tag_seperate(testing_set)
  all_test_tags.append(test_tags)
  vocab, tags, vocab_word_tag = get_tag_vocab(training_set)
  print(len(test_words))

  transition_counts, emission_counts, tag_counts = get_counts(training_set)
  two_tag_counts = get_2_tag_counts(training_set)
  print(two_tag_counts)
  print(transition_counts)
  print(emission_counts)
  transition_probs = get_transition_matrix(transition_counts, two_tag_counts, tags)
  emission_probs = get_emission_matrix(emission_counts, tag_counts, tags, vocab)
  print(transition_probs)
  print(emission_probs)
  predicted_tags.append(viterbi(transition_probs, emission_probs, test_words, vocab_word_tag))
  # break

NameError: ignored

In [ ]:
def accuracy(actual, pred):
  n = len(actual)
  correct = 0
  for i in range(n):
    if actual[i] == pred[i]:
      correct += 1
    # else:
    #   print(actual[i], pred[i])
  return correct/n

In [ ]:
for i in range(3):
  print(accuracy(all_test_tags[i], predicted_tags[i]))
  break

In [ ]:
save_all_test_tags = np.array(all_test_tags)
save_predicted_tags = np.array(predicted_tags)

In [ ]:
np.save('save_all_test_tags', save_all_test_tags)
np.save('save_predicted_tags', save_predicted_tags)
# np.save('save_transition_counts', save_transition_counts)
# np.save('save_emission_counts', save_emission_counts)
# np.save('save_tag_counts', save_tag_counts)
# np.save('save_two_tag_counts', save_two_tag_counts)

In [ ]:
!cp save_all_test_tags.npy "drive/My Drive/NLP"
!cp save_predicted_tags.npy "drive/My Drive/NLP"
# !cp save_transition_counts.npy "drive/My Drive/NLP"
# !cp save_emission_counts.npy "drive/My Drive/NLP"
# !cp save_tag_counts.npy "drive/My Drive/NLP"
# !cp save_two_tag_counts.npy "drive/My Drive/NLP"

In [ ]:
TEST_TAG = np.load('drive/My Drive/NLP/save_all_test_tags.npy')
PRED_TAG = np.load('drive/My Drive/NLP/save_all_test_tags.npy')

In [ ]:
def performance_metrics(actual, pred, tags, num):
  # print(len(actual), len(pred), len(tags))
  df = pd.DataFrame(0, columns = tags, index = tags) # columns are true tags, rows are predicted tags
  # print(df)
  n = len(actual)
  for i in range(n):
    try:
      df[actual[i]][pred[i]] += 1
    except KeyError:
      pass
    # if i%100==0:
    #   print('Processed {}'.format(i))
  # print('Confusion Matrix')
  # print(df)
  df.to_csv(str(num) + '_confusion_matrix.csv')
  precision = {}
  recall = {}
  predicted_sum = df.sum(axis=0)
  actual_sum = df.sum(axis=1)
  for tag in tags:
    precision[tag] = [df[tag][tag]/predicted_sum[tag]]
    recall[tag] = [df[tag][tag]/actual_sum[tag]]
  f1 = {}
  for tag in tags:
    f1[tag] = [harmonic_mean([precision[tag][0], recall[tag][0]])]
  # print('Tagwise precision', precision)
  # print('Tagwise recall', recall)
  # print('Tagwise F1 score', f1)
  df_precision = pd.DataFrame.from_dict(precision) 
  df_recall = pd.DataFrame.from_dict(recall) 
  df_f1 = pd.DataFrame.from_dict(f1) 
  df_precision.to_csv(str(num) + '_precision.csv')
  df_recall.to_csv(str(num) + '_recall.csv')
  df_f1.to_csv(str(num) + '_f1.csv')

  overall_precision = df_precision.stack().mean()
  overall_recall = df_recall.stack().mean()
  overall_f1 = df_f1.stack().mean()

  print('Precision', overall_precision)
  print('Recall', overall_recall)
  print('F1 score', overall_f1)

In [ ]:
for i in range(3):
  print('Fold', i+1)
  performance_metrics(all_test_tags[i], predicted_tags[i], full_tags, i)